In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('Film_parsing.csv')
df.set_index('Unnamed: 0', inplace = True)

df.head()

,date_added,views,comments,Title,Text
Unnamed: 0,,,,,
0,2015-12-07,55462,70,Настоящий детектив,"Небольшой промышленный город Винчи, штат Калиф..."
1,2014-11-10,80165,99,Настоящий детектив,"Главные герои сериала - детектив Раст ""Ра..."
2,2016-12-22,40431,46,Хорас и Пит,"""У Хораса и Пита"" - старинный ирландский паб в..."
3,2020-11-30,103921,287,Ход королевы,"Америка, середина пятидесятых. Девятилетняя де..."
4,2020-09-14,74243,29,Рома,"Мексика, город Мехико, 1970 год. Богатый район..."


# Предобработка данных

### Токенизация и удаление стоп-слов 

In [3]:
# Загружаем стоп-слова для русского языка

import nltk

nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('russian')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Даниил\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# Инициализируем токенайзер

word_tokenizer = nltk.WordPunctTokenizer()

In [101]:
# Функция отбора слов

import re
regex = re.compile(r'[А-Яа-яA-zёЁ]+')

def words_only(text, regex = regex):
    try:
        return ' '.join(regex.findall(text)).lower()
    except:
        return ''

In [102]:
# Функция первичной предобработки слов

def process_data(data):
    texts = []
    targets = []
    
    for item in data:
        
        # оставим только слова
        text_lower = words_only(item)
        tokens = word_tokenizer.tokenize(text_lower)
        
        # удаляем пунктуацию и стоп-слова
        tokens = [word for word in tokens if (word not in stop_words and not word.isnumeric())]
        
        texts.append(tokens)
        
    return texts

In [103]:
# Запуск предобработки текста

text = process_data(df['Text'])

In [104]:
# Результат предобработки

i = 1
print('Слова: ', text[i][:6])

Слова:  ['главные', 'герои', 'сериала', 'детектив', 'раст', 'расти']


### Нормализация слов 

Для русского применим лематизацию

In [105]:
import pymorphy2
from tqdm import tqdm_notebook

morph = pymorphy2.MorphAnalyzer()

for i in tqdm_notebook(range(len(text)), desc = 'Word'):
    text_lemmatized = [morph.parse(x)[0].normal_form for x in text[i]]
    text[i] = ' '.join(text_lemmatized)

Word:   0%|          | 0/1290 [00:00<?, ?it/s]

In [142]:
# Разделим обработанный текст по словам

texts = [text[i].split() for i in range(len(text))]


In [71]:
len(texts)

1290

# Векторизируем слова Word2Vec

In [72]:
# Инициализация Word2Vec

import multiprocessing
from gensim.models import Word2Vec

cores = multiprocessing.cpu_count()

In [73]:
w2v_model = Word2Vec(texts,
                     min_count = 5,
                     window = 5,
                     vector_size = 500,
                     sample = 0.00006,
                     min_alpha = 0.0007,
                     negative = cores - 1)

In [74]:
# Похожие слова на фильм

w2v_model.wv.most_similar('фильм')

[('посмотреть', 0.9928498268127441),
 ('картина', 0.9888120889663696),
 ('рейтинг', 0.9856679439544678),
 ('imdb', 0.9833210706710815),
 ('очень', 0.9817618131637573),
 ('s', 0.978566586971283),
 ('смотреть', 0.9773574471473694),
 ('хороший', 0.9767132997512817),
 ('понравиться', 0.9763575196266174),
 ('дубляж', 0.9754841923713684)]

In [75]:
# Похожие слова на продюсер

w2v_model.wv.most_similar('продюсер')

[('ставить', 0.9995913505554199),
 ('проект', 0.9995712637901306),
 ('качество', 0.9995290637016296),
 ('создание', 0.9995121955871582),
 ('популярный', 0.9995061755180359),
 ('экранизация', 0.9994924664497375),
 ('съёмка', 0.9994916319847107),
 ('нашуметь', 0.9994905591011047),
 ('решить', 0.9994863867759705),
 ('известный', 0.9994707107543945)]

In [76]:
# Похожие слова на полицейский

w2v_model.wv.most_similar('полицейский')

[('начальник', 0.9996540546417236),
 ('приехать', 0.9996282458305359),
 ('дочка', 0.999606192111969),
 ('приезжать', 0.9996060729026794),
 ('полиция', 0.9995954632759094),
 ('девушка', 0.9995923638343811),
 ('молодой', 0.9995884895324707),
 ('дочь', 0.9995861053466797),
 ('сбежать', 0.9995858669281006),
 ('преступник', 0.9995781779289246)]

In [77]:
# режиссёр - фильм

vec = (w2v_model.wv['режиссёр'] - w2v_model.wv['фильм'])
w2v_model.wv.similar_by_vector(vec)

[('свой', 0.2288573682308197),
 ('дом', 0.2103424370288849),
 ('жена', 0.20517925918102264),
 ('жить', 0.20168359577655792),
 ('сын', 0.19789811968803406),
 ('отец', 0.19615104794502258),
 ('нью', 0.19513772428035736),
 ('местный', 0.1948694884777069),
 ('семья', 0.19299384951591492),
 ('йорк', 0.19294701516628265)]

In [78]:
# мама - папа

vec = (w2v_model.wv['мама'] - w2v_model.wv['папа'])
w2v_model.wv.similar_by_vector(vec)

[('аж', 0.9238555431365967),
 ('приз', 0.922783613204956),
 ('номинировать', 0.9226051568984985),
 ('получить', 0.9224628210067749),
 ('победа', 0.9216273427009583),
 ('год', 0.921438455581665),
 ('семь', 0.9211888909339905),
 ('шесть', 0.9210574626922607),
 ('bafta', 0.9210347533226013),
 ('мужской', 0.9210277795791626)]

In [79]:
# Лишнее слово

w2v_model.wv.doesnt_match('герой картина друг персонаж'.split())

'картина'

# NER 

In [52]:
# Импортируем библиотеки для русского языка

from navec import Navec
from slovnet import NER
from ipymarkup import show_span_box_markup as show_markup

# Скачаем предобученные эмбединги 
navec = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')

# Скачаем предобученную модель

ner = NER.load('slovnet_ner_news_v1.tar')

In [53]:
# Передадим данные в предобученные модели

ner.navec(navec)

markup = ner(df.loc[0, 'Text'])

In [55]:
# Визуализация именнованных сущностей

show_markup(df.loc[0, 'Text'], markup.spans)

# Тематическое моделирование

In [80]:
# Дополнительная предобработка текста для тематического мо делирования

from gensim.models import Phrases
from gensim.corpora import Dictionary

bigram = Phrases(texts, min_count = 10) # создадим биграммы слов

for idx in range(len(texts)):
    for token in bigram[texts[idx]]:
        if '_' in token:
        # Добавим биграммы в основной обработанный корпус
            texts[idx].append(token)

In [81]:
texts[0]

['небольшой',
 'промышленный',
 'город',
 'винчи',
 'штат',
 'калифорния',
 'городок',
 'происходить',
 'зверский',
 'убийство',
 'видный',
 'чиновник',
 'городской',
 'администрация',
 'имя',
 'рэй',
 'каспар',
 'убитый',
 'след',
 'пытка',
 'обнаруживать',
 'город',
 'офицер',
 'дорожный',
 'служба',
 'пол',
 'вудро',
 'тэйлор',
 'китч',
 'место',
 'происшествие',
 'вызывать',
 'полицейский',
 'близкий',
 'городок',
 'штат',
 'детектив',
 'ани',
 'беззеридес',
 'рэйчел',
 'макадамс',
 'детектив',
 'рэй',
 'велкоро',
 'колин',
 'фаррелла',
 'это',
 'убийство',
 'произойти',
 'момент',
 'винчи',
 'должный',
 'представить',
 'проект',
 'новый',
 'скоростной',
 'железный',
 'дорога',
 'соединять',
 'северный',
 'южный',
 'часть',
 'штат',
 'проект',
 'контролировать',
 'мэр',
 'город',
 'остин',
 'чессани',
 'ричать',
 'костёр',
 'видный',
 'бизнесмен',
 'фрэнк',
 'семьон',
 'винс',
 'вон',
 'который',
 'начинать',
 'организовать',
 'преступность',
 'владелец',
 'казино',
 'всякий',
 'зл

In [83]:
dictionary = Dictionary(texts)
dictionary.filter_extremes(no_below=10, no_above = 0.3)
vars(dictionary)

{'token2id': {'авторитет': 0,
  'администрация': 1,
  'азартный': 2,
  'азартный_игра': 3,
  'актёрский': 4,
  'актёрский_игра': 5,
  'акцент': 6,
  'александр': 7,
  'алкоголить': 8,
  'алкоголь': 9,
  'аллен': 10,
  'анджелес': 11,
  'армянский': 12,
  'асоциальный': 13,
  'ах': 14,
  'бандит': 15,
  'бандитский': 16,
  'беец': 17,
  'безнадёга': 18,
  'безэмоциональный': 19,
  'белый': 20,
  'бескрайний': 21,
  'беспощадный': 22,
  'бесстрашный': 23,
  'бизнес': 24,
  'бизнесмен': 25,
  'благодаря': 26,
  'близкий': 27,
  'бой': 28,
  'брендать': 29,
  'брендать_глисон': 30,
  'бровка': 31,
  'буквально': 32,
  'бульдог': 33,
  'бутылка': 34,
  'важный': 35,
  'вводиться': 36,
  'ведущий': 37,
  'великий': 38,
  'вестись': 39,
  'вечер': 40,
  'вечеринка': 41,
  'взаимоотношение': 42,
  'взгляд_получиться': 43,
  'взятка': 44,
  'видеться': 45,
  'видно': 46,
  'видный': 47,
  'винс': 48,
  'виски': 49,
  'владелец': 50,
  'власть': 51,
  'влезть': 52,
  'влиять': 53,
  'вложить': 5

In [89]:
# Закодируем слова в корпусе текста и добавим частоту

corpus = [dictionary.doc2bow(text) for text in texts]
corpus[0]

[(0, 1),
 (1, 2),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 3),
 (10, 1),
 (11, 2),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 3),
 (16, 1),
 (17, 1),
 (18, 2),
 (19, 1),
 (20, 1),
 (21, 1),
 (22, 1),
 (23, 1),
 (24, 2),
 (25, 2),
 (26, 1),
 (27, 2),
 (28, 1),
 (29, 1),
 (30, 1),
 (31, 1),
 (32, 1),
 (33, 1),
 (34, 1),
 (35, 3),
 (36, 1),
 (37, 1),
 (38, 1),
 (39, 2),
 (40, 1),
 (41, 1),
 (42, 2),
 (43, 1),
 (44, 2),
 (45, 1),
 (46, 2),
 (47, 2),
 (48, 4),
 (49, 1),
 (50, 2),
 (51, 1),
 (52, 1),
 (53, 1),
 (54, 1),
 (55, 1),
 (56, 1),
 (57, 2),
 (58, 1),
 (59, 2),
 (60, 1),
 (61, 5),
 (62, 1),
 (63, 1),
 (64, 1),
 (65, 1),
 (66, 1),
 (67, 1),
 (68, 1),
 (69, 1),
 (70, 10),
 (71, 3),
 (72, 2),
 (73, 1),
 (74, 1),
 (75, 2),
 (76, 1),
 (77, 1),
 (78, 1),
 (79, 3),
 (80, 1),
 (81, 1),
 (82, 1),
 (83, 3),
 (84, 1),
 (85, 1),
 (86, 6),
 (87, 2),
 (88, 2),
 (89, 1),
 (90, 2),
 (91, 1),
 (92, 1),
 (93, 1),
 (94, 1),
 (95, 1),
 (96, 1),
 (97, 1),
 (98, 2),
 (99, 1),
 (100, 1)

In [85]:
print('Количество уникальных токенов: %d' % len(dictionary))
print('количество документов: %d' % len(corpus))

Количество уникальных токенов: 10815
количество документов: 1290


In [134]:
# Обучим LDA модель

from gensim.models import LdaModel
import gensim


# id2word = dictionary.id2token

model = gensim.models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    chunksize=600,
    eta='auto',
    iterations=400,
    num_topics=50,
    passes=40,
    eval_every=None)

In [135]:
# отберём кол-во слов для терминов

topics = model.top_topics(corpus, topn = 50)

In [136]:
# Посмотрим на 4 тему

topics[4]

([(0.0121494625, 'джо'),
  (0.007627902, 'номинация'),
  (0.0054363795, 'проект'),
  (0.0053704777, 'постановка'),
  (0.005184704, 'оскар'),
  (0.0051123407, 'миссис'),
  (0.0045760185, 'заметить'),
  (0.004364631, 'актриса'),
  (0.0038355975, 'звезда'),
  (0.0037982902, 'важный'),
  (0.0037958224, 'уильямс'),
  (0.0037839562, 'награда'),
  (0.0037069684, 'миллер'),
  (0.0036991963, 'премия'),
  (0.00364406, 'чувство'),
  (0.0035370463, 'собираться'),
  (0.0035368598, 'спектакль'),
  (0.0034824626, 'музыка'),
  (0.0033425896, 'студия'),
  (0.00328961, 'джаз'),
  (0.0032782268, 'идея'),
  (0.0032238613, 'эмоция'),
  (0.003214314, 'золотой'),
  (0.0032074526, 'богатый'),
  (0.0031551963, 'любовь'),
  (0.0030808332, 'съёмка'),
  (0.0030563762, 'четыре'),
  (0.0030205518, 'сниматься'),
  (0.002982633, 'месяц'),
  (0.0029581294, 'изображать'),
  (0.0029398636, 'взаимоотношение'),
  (0.0027979636, 'неделя'),
  (0.0027331172, 'отправляться'),
  (0.0027002834, 'музыкальный'),
  (0.0026776395, 

In [137]:
# Посмотрим на другую тему

topics[7]

([(0.032034237, 'рита'),
  (0.02660562, 'гонка'),
  (0.025625031, 'хант'),
  (0.025400745, 'формула'),
  (0.012253584, 'хоуард'),
  (0.011774025, 'машина'),
  (0.010620415, 'гонщик'),
  (0.00952098, 'пилот'),
  (0.008931181, 'команда'),
  (0.008883217, 'даниэль'),
  (0.008267027, 'ника'),
  (0.007594484, 'победа'),
  (0.0073244874, 'чемпион'),
  (0.006561081, 'немец'),
  (0.0064279805, 'спортивный'),
  (0.0059825378, 'реальный'),
  (0.0056204274, 'характер'),
  (0.0054893545, 'выступать'),
  (0.0052368436, 'противостояние'),
  (0.005072911, 'винс'),
  (0.004905691, 'руководство'),
  (0.0047829286, 'сходство'),
  (0.004678934, 'лорд'),
  (0.00467205, 'м'),
  (0.004663844, 'существовать'),
  (0.0046548015, 'кадр'),
  (0.0045709494, 'соревнование'),
  (0.0043620453, 'оба'),
  (0.0043298863, 'скрупулёзно'),
  (0.0043293736, 'добиться'),
  (0.0041885716, 'точно'),
  (0.0041771866, 'уйти'),
  (0.0041492097, 'автомобиль'),
  (0.004053332, 'попасть'),
  (0.004013019, 'добиваться'),
  (0.003995

In [138]:
# Посмотрим на распределение тем у 1 отзыва 

vector = model[corpus[0]]

vector

[(1, 0.019069523),
 (2, 0.012145464),
 (3, 0.011655235),
 (4, 0.0130880885),
 (6, 0.042073376),
 (7, 0.031328943),
 (10, 0.22780326),
 (13, 0.08922656),
 (15, 0.051992193),
 (17, 0.04161977),
 (18, 0.017690834),
 (23, 0.018958287),
 (25, 0.02027688),
 (27, 0.0215445),
 (29, 0.02860397),
 (30, 0.022783415),
 (31, 0.09477656),
 (34, 0.053340454),
 (41, 0.033038907),
 (42, 0.046212975),
 (43, 0.04173043),
 (44, 0.041941065),
 (49, 0.018428326)]

Видно что 0 отзыву больше всего подходит 10 тема

In [126]:
print(text[0])

небольшой промышленный город винчи штат калифорния городок происходить зверский убийство видный чиновник городской администрация имя рэй каспар убитый след пытка обнаруживать город офицер дорожный служба пол вудро тэйлор китч место происшествие вызывать полицейский близкий городок штат детектив ани беззеридес рэйчел макадамс детектив рэй велкоро колин фаррелла это убийство произойти момент винчи должный представить проект новый скоростной железный дорога соединять северный южный часть штат проект контролировать мэр город остин чессани ричать костёр видный бизнесмен фрэнк семьон винс вон который начинать организовать преступность владелец казино всякий злачный заведение решить связаться преступность крутой поэтому завязать деловой связь мэр чессани вложиться покупка земля который пройти строительство сулить весь владелец земля больший прибыль также фрэнк собираться заключить больший сделка крупный русский авторитет осип агроновое тимоти ви мерфь который обещать вложить проект семьон кру

In [139]:
topics[10]

([(0.0115243, 'александр'),
  (0.011454929, 'российский'),
  (0.01059647, 'куценко'),
  (0.009667815, 'сергей'),
  (0.009507518, 'максим'),
  (0.007745532, 'читать'),
  (0.0076504066, 'гоша'),
  (0.0068917815, 'юрий'),
  (0.0062876954, 'евгений'),
  (0.0061867945, 'милиция'),
  (0.0059074387, 'светлый'),
  (0.0057967287, 'сила'),
  (0.005729681, 'михаил'),
  (0.005729377, 'андрей'),
  (0.0057049375, 'гоша_куценко'),
  (0.005654693, 'анна'),
  (0.005253133, 'марина'),
  (0.005127532, 'москва'),
  (0.0051117353, 'рецензия'),
  (0.0051019494, 'папа'),
  (0.0050760442, 'роман'),
  (0.004989795, 'бадди'),
  (0.004774877, 'критика'),
  (0.0047708713, 'явный'),
  (0.004713522, 'мария'),
  (0.0047087497, 'город'),
  (0.004438373, 'оружие'),
  (0.0044143195, 'дмитрий'),
  (0.0043448983, 'владимир'),
  (0.004223965, 'отзыв'),
  (0.0040976238, 'диалог'),
  (0.004009584, 'дурацкий'),
  (0.0037728357, 'критик'),
  (0.0037627036, 'дескать'),
  (0.0034903348, 'юлия'),
  (0.0034800703, 'милиционер'),
